1. 문서의 내용을 읽는다. 
2. 문서를 쪼갠다. 
    - 토큰 수 초과로 답변을 생성하지 못할 수 있고 
    - 문서가 길면 (input 이 길면) 답변 생성이 오래걸림 
3. 임베딩 -> 벡터 데이터베이스에 저장 
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색 
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 

In [1]:
%pip install --upgrade --quiet docx2txt langchain-community


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import Docx2txtLoader

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [3]:
%pip install -qU langchain-text-splitter

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

loader = Docx2txtLoader("./tax.docx")
document_list = loader.load_and_split(text_splitter=text_splitter)


In [5]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


In [7]:
%pip install -qU langchain langchain-pinecone langchain-openai


[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os 
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

load_dotenv()

index_name = "tax-index"
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(
    document_list,
    embeddings,
    index_name=index_name
)

In [13]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3)

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [15]:
from langchain import hub 

prompt = hub.pull("rlm/rag-prompt")

/Users/suuuheee/.pyenv/versions/llm-application/lib/python3.10/site-packages/langsmith/client.py:288: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [16]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [17]:
ai_message = qa_chain({"query": query})

/var/folders/d1/9_vjddxd3_l9_cgptdtp88gm0000gn/T/ipykernel_17369/3455095564.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ai_message = qa_chain({"query": query})


In [18]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '해당 문서는 구체적인 소득세 부과 기준이나 계산 방법에 관한 정보를 제공하고 있지 않습니다. 연봉 5천만 원인 직장인의 소득세를 계산하려면 해당 연도의 소득세율과 공제 항목을 참고하여야 합니다. 정확한 금액을 알기 위해서는 국세청의 소득세 계산기를 활용하는 방법이 있습니다.'}